# Demonstrate Git Based ML Pipeline Automation
  --------------------------------------------------------------------

Creating a local function, running predefined functions, creating and running a full ML pipeline with local and library functions.

#### **notebook how-to's**
* Create and test a simple function
* Examine data using serverless (containarized) `describe` function
* Create an automated ML pipeline from various library functions
* Running and tracking the pipeline results and artifacts

## Create and Test a Local Ingestion/Data-prep Function (e.g. Iris Data Generator)
Import nuclio SDK and magics, <b>do not remove the cell and comment !!!</b>

In [1]:
# nuclio: ignore
import nuclio

<b>Specify function dependencies and configuration<b>

In [2]:
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting spec.image to 'mlrun/ml-models'


#### Function code
Generate the iris dataset and log the dataframe (as csv or parquet file)

In [3]:
import os
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from mlrun.artifacts import TableArtifact, PlotArtifact
import pandas as pd

def iris_generator(context, format='csv'):
    iris = load_iris()
    iris_dataset = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    iris_labels = pd.DataFrame(data=iris.target, columns=['label'])
    iris_dataset = pd.concat([iris_dataset, iris_labels], axis=1)
    
    context.logger.info('saving iris dataframe to {}'.format(context.artifact_path))
    context.log_dataset('iris_dataset', df=iris_dataset, format=format, index=False)


The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [4]:
# nuclio: end-code
# marks the end of a code section

## Create a project to host our functions, jobs and artifacts

Projects are used to package multiple functions, workflows, and artifacts. We usually store project code and definitions in a Git archive.

The following code creates a new project in a local dir and initialize git tracking on that

In [5]:
from os import path
from mlrun import run_local, NewTask, mlconf, import_function, mount_v3io
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

# specify artifacts target location
artifact_path = mlconf.artifact_path or path.abspath('./')
project_name = 'gitops-project'

In [6]:
from mlrun import new_project, code_to_function
project_dir = './'
skproj = new_project(project_name, project_dir)

<a id='test-locally'></a>
### Run/test the data generator function locally

The functions above can be tested locally. Parameters, inputs, and outputs can be specified in the API or the `Task` object.<br>
when using `run_local()` the function inputs and outputs are automatically recorded by MLRun experiment and data tracking DB.

In each run we can specify the function, inputs, parameters/hyper-parameters, etc... For more details, see the [mlrun_basics notebook](mlrun_basics.ipynb).

In [7]:
# run the function locally
gen = run_local(name='iris_gen', handler=iris_generator, 
                project=project_name, artifact_path=path.join(artifact_path, 'data')) 

> 2021-07-15 23:48:33,150 [info] starting run iris_gen uid=c956bbcbedee41ddbae86cc2ec4a4769 DB=http://mlrun-api:8080
> 2021-07-15 23:48:33,261 [info] saving iris dataframe to v3io:///projects/gitops-project/artifacts/data


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
gitops-project,...ec4a4769,0,Jul 15 23:48:33,completed,iris_gen,v3io_user=xingshengkind=handlerowner=xingshenghost=jupyter-xingsheng-5d74b7949c-grgfg,,,,iris_dataset


to track results use .show() or .logs() or in CLI: 
!mlrun get run c956bbcbedee41ddbae86cc2ec4a4769 --project gitops-project , !mlrun logs c956bbcbedee41ddbae86cc2ec4a4769 --project gitops-project
> 2021-07-15 23:48:33,458 [info] run executed, status=completed


#### Convert our local code to a distributed serverless function object 

In [8]:
gen_func = code_to_function(name='gen_iris', kind='job')
skproj.set_function(gen_func)

## Analyze the dataset features (useing marketplace function)
load dataset analysis function (`describe`) from the function hub (marketplace), and print its doc.

In [9]:
skproj.set_function('hub://describe', 'describe')
skproj.func('describe').doc()

function: describe
describe and visualizes dataset stats
default handler: summarize
entry points:
  summarize: Summarize a table
    context(MLClientCtx)  - the function context, default=
    table(DataItem)  - MLRun input pointing to pandas dataframe (csv/parquet file path), default=
    label_column(str)  - ground truth column label, default=None
    class_labels(List[str])  - label for each class in tables and plots, default=[]
    plot_hist(bool)  - (True) set this to False for large tables, default=True
    plots_dest(str)  - destination folder of summary plots (relative to artifact_path), default=plots
    update_dataset  - when the table is a registered dataset update the charts in-place, default=False


### Run the describe function on our dataset (as a Kubernetes job)
<b> using shared file system mount (`mount_v3io`) with our notebook.</b>

In [10]:
skproj.func('describe').apply(mount_v3io()).run(params={'label_column': 'label'}, 
                                                inputs={"table": gen.outputs['iris_dataset']}, 
                                                artifact_path=artifact_path)

> 2021-07-15 23:48:41,841 [info] starting run describe-summarize uid=22e9a25bd67447b59a205c7c1acec99d DB=http://mlrun-api:8080
> 2021-07-15 23:48:42,120 [info] Job is running in the background, pod: describe-summarize-z54xj
> 2021-07-15 23:48:52,939 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
gitops-project,...1acec99d,0,Jul 15 23:48:47,completed,describe-summarize,v3io_user=xingshengkind=jobowner=xingshenghost=describe-summarize-z54xj,table,label_column=label,,histogramsviolinimbalanceimbalance-weights-veccorrelation-matrixcorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run 22e9a25bd67447b59a205c7c1acec99d --project gitops-project , !mlrun logs 22e9a25bd67447b59a205c7c1acec99d --project gitops-project
> 2021-07-15 23:49:01,401 [info] run executed, status=completed


## Create a Fully Automated ML Pipeline

#### Add more functions to our project to be used in our pipeline (from the functions hub/marketplace)

AutoML training (classifier), Model validation (test_classifier), Real-time model server, and Model REST API Tester

In [11]:
skproj.set_function('hub://sklearn_classifier', 'train')
skproj.set_function('hub://test_classifier', 'test')
skproj.set_function('hub://model_server', 'serving')
skproj.set_function('hub://model_server_tester', 'live_tester')
#print(skproj.to_yaml())

#### Define and save a pipeline 

The following workflow definition will be written into a file, it describes a Kubeflow execution graph (DAG)<br>
and how functions and data are connected  to form an end to end pipeline. 

* Build the iris generator (ingest) function container 
* Ingest the iris data
* Analyze the dataset (describe)
* Train and test the model
* Deploy the model as a real-time serverless function
* Test the serverless function REST API with test dataset

Check the code below to see how functions objects are initialized and used (by name) inside the workflow.<br>
The `workflow.py` file has two parts, initialize the function objects and define pipeline dsl (connect the function inputs and outputs).

> Note: the pipeline can include CI steps like building container images and deploying models as illustrated  in the following example.


In [12]:
%%writefile ./workflow.py
from kfp import dsl
from mlrun import mount_v3io, NewTask


funcs = {}
this_project = None
DATASET = 'iris_dataset'
LABELS  = "label"

# init functions is used to configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())
     
    # uncomment this line to collect the inference results into a stream
    # and specify a path in V3IO (<datacontainer>/<subpath>)
    #functions['serving'].set_env('INFERENCE_STREAM', 'users/admin/model_stream')

    
@dsl.pipeline(
    name="Demo training pipeline",
    description="Shows how to use mlrun."
)
def kfpipeline():
    
    # run the ingestion function with the new image and params
    ingest = funcs['gen-iris'].as_step(
        name="get-data",
        handler='iris_generator',
        params={'format': 'pq'},
        outputs=[DATASET])

    # analyze our dataset
    describe = funcs["describe"].as_step(
        name="summary",
        params={"label_column": LABELS},
        inputs={"table": ingest.outputs[DATASET]})
    
    # train with hyper-paremeters
    train = funcs["train"].as_step(
        name="train",
        params={"sample"          : -1,
                "label_column"    : LABELS,
                "test_size"       : 0.10},
        hyperparams={'model_pkg_class': ["sklearn.ensemble.RandomForestClassifier",
                                         "sklearn.linear_model.LogisticRegression",
                                         "sklearn.ensemble.AdaBoostClassifier"]},
        selector='max.accuracy',
        inputs={"dataset"         : ingest.outputs[DATASET]},
        labels={"commit": this_project.params.get('commit', '')},
        outputs=['model', 'test_set'])

    # test and visualize our model
    test = funcs["test"].as_step(
        name="test",
        params={"label_column": LABELS},
        inputs={"models_path" : train.outputs['model'],
                "test_set"    : train.outputs['test_set']})

    # deploy our model as a serverless function
    deploy = funcs["serving"].deploy_step(models={f"{DATASET}_v1": train.outputs['model']},
                                          tag=this_project.params.get('commit', 'v1'))

    # test out new model server (via REST API calls)
    tester = funcs["live_tester"].as_step(name='model-tester',
        params={'addr': deploy.outputs['endpoint'], 'model': f"{DATASET}_v1"},
        inputs={'table': train.outputs['test_set']})


Overwriting ./workflow.py


In [13]:
# register the workflow file as "main", embed the workflow code into the project YAML
skproj.set_workflow('main', 'workflow.py')

Save the project definitions to a file (project.yaml), it is recommended to commit all changes to a Git repo.

In [14]:
skproj.artifact_path = 'v3io:///users/{{run.user}}/pipe/{{workflow.uid}}'
skproj.save()

<a id='run-pipeline'></a>
## Run a pipeline workflow manually (not via git PR)

<b>This section is not used for the git automation, rather demo how to run the workflow from the notebook</b>

use the `run` method to execute a workflow, you can provide alternative arguments and specify the default target for workflow artifacts.<br>
The workflow ID is returned and can be used to track the progress or you can use the hyperlinks

> Note: The same command can be issued through CLI commands:<br>
    `mlrun project my-proj/ -r main -p "v3io:///users/{{run.user}}/mlrun/kfp/{{workflow.uid}}/"`

The `dirty` flag allow us to run a project with uncommited changes (when the notebook is in the same git dir it will always be dirty)<br>
The `watch` flag will wait for the pipeline to complete

In [18]:
#!mlrun project . -r main -p "v3io:///users/{{run.user}}/mlrun/kfp/{{workflow.uid}}/"

Loading project gitops-project into .:

kind: project
metadata:
  name: gitops-project
spec:
  functions:
  - name: gen-iris
    spec:
      kind: job
      metadata:
        name: gen-iris
        tag: ''
        project: gitops-project
      spec:
        command: ''
        args: []
        image: mlrun/ml-models
        env: []
        default_handler: ''
        entry_points:
          iris_generator:
            name: iris_generator
            doc: ''
            parameters:
            - name: context
              default: ''
            - name: format
              default: csv
            outputs:
            - default: ''
            lineno: 11
        description: ''
        build:
          functionSourceCode: IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnRlcgoKaW1wb3J0IG9zCmZyb20gc2tsZWFybi5kYXRhc2V0cyBpbXBvcnQgbG9hZF9pcmlzCmZyb20gc2tsZWFybi5tb2RlbF9zZWxlY3Rpb24gaW1wb3J0IHRyYWluX3Rlc3Rfc3BsaXQKaW1wb3J0IG51bXB5IGFzIG5wCmZyb20gc2tsZWFybi5tZXRyaWNzIGltcG9ydCBhY2N1cm

In [15]:
# If you want to get slack notification after the run with result summary, set the env var below
# %env SLACK_WEBHOOK=<slack webhooh url>

In [16]:
run_id = skproj.run(
    'main', arguments={}, 
    dirty=True, watch=True)

> 2021-07-15 23:49:02,400 [info] using in-cluster config.


> 2021-07-15 23:49:03,018 [info] Pipeline run id=3b2706ca-253d-40fc-802d-a43eec952abb, check UI or DB for progress
> 2021-07-15 23:49:03,019 [info] waiting for pipeline run completion


uid,start,state,name,results,artifacts
...e65ace3d,Jul 15 23:50:00,completed,model-tester,total_tests=15errors=0match=14avg_latency=116849min_latency=15925max_latency=991106,latency
...e8b1f01e,Jul 15 23:49:46,completed,test,accuracy=0.9333333333333333test-error=0.06666666666666667auc-micro=0.9755555555555555auc-weighted=0.9944444444444444f1-score=0.9137254901960784precision_score=0.8888888888888888recall_score=0.9629629629629629,confusion-matrixfeature-importancesprecision-recall-multiclassroc-multiclasstest_set_preds
...d91d36e2,Jul 15 23:49:25,completed,summary,,histogramsviolinimbalanceimbalance-weights-veccorrelation-matrixcorrelation
...7e6d1f0f,Jul 15 23:49:24,completed,train,best_iteration=1accuracy=0.975609756097561test-error=0.024390243902439025auc-micro=0.9973230220107078auc-weighted=0.9966358284272497f1-score=0.9721739130434783precision_score=0.9743589743589745recall_score=0.9722222222222222,test_setconfusion-matrixfeature-importancesprecision-recall-multiclassroc-multiclassmodeliteration_results
...0c75f25e,Jul 15 23:49:12,completed,get-data,,iris_dataset


**[back to top](#top)**